Реализация PINN на pytorch взята отсюда: https://github.com/jayroxis/PINNs/blob/master/Burgers%20Equation/Burgers%20Inference%20(PyTorch).ipynb

#Уравнение второго порядка:

коэффициенты

In [ ]:
import math
alpha = 0.0
beta = 0.
k=2.0
w=1.7
x0=0.0
th0=0.0

решение

In [ ]:
def q(x,t):
  kexp = np.exp(np.sqrt(k ** 2 - w) * (x - 2 * k * t - x0))
  f_com = 4 * (k ** 2 - w) / (kexp + 2 * (k ** 2 - w) / kexp)
  u = f_com * np.cos(k * x - w * t + th0)
  v = f_com * np.sin(k * x - w * t + th0)
  return u, v

In [ ]:
!pip install pyDOE

In [ ]:
import torch
from collections import OrderedDict #упорядоченный словарь
from pyDOE import lhs #функция, выбирающая значения для обучения на них нейросети
import numpy as np
import time
np.random.seed(1234)

In [ ]:
#можно сменить среду выполнения на gpu и обучение будет происходить быстрее
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
class SinActivation(torch.nn.Module): #кастомная функция активации - sin
    def __init__(self):
        super(SinActivation, self).__init__()
        return
    def forward(self, x):
        return torch.sin(x)

class FourierTransform(torch.nn.Module):
  def __init__(self):
    super(FourierTransform, self).__init__()
    return
  def forward(self, X):
    return torch.fft.rfftn(X)

class InverseFourierTransform(torch.nn.Module):
  def __init__(self):
    super(InverseFourierTransform, self).__init__()
    return
  def forward(self, X):
    return torch.fft.irfftn(X)

нейросеть, в виде которой будет находиться решение

In [ ]:
class FDNN(torch.nn.Module):
    def __init__(self, layers): #принимает на вход массив целых чисел
        super(FDNN, self).__init__() #вызывает метод init(почему нельзя сделать это без super?)

        self.depth = len(layers) - 1
        self.activation = SinActivation #в качестве функции активации используется sin
        self.ft = FourierTransform #прямое дискретное преобразование Фурье
        self.ift = InverseFourierTransform # обратное дискретное преобразование Фурье

        dimension_list = [0]*self.depth #лист с размерностью каждого слоя, нужно для правильной обработки 'FL'
        for i in range(self.depth):
          if (isinstance(layers[i], int)): dimension_list[i]=layers[i]
          elif (layers[i]=='FL'): dimension_list[i]=dimension_list[i-1]

        layer_list = list() #список с весами и функциями активации каждого слоя
        for i in range(self.depth - 1):
          if (isinstance(layers[i], int)):
            layer_list.append(('layer_%d' % i, torch.nn.Linear(dimension_list[i], dimension_list[i+1]))) #каждые два слоя образуют двудольный граф
            layer_list.append(('activation_%d' % i, self.activation()))
          elif (layers[i]=='FL'):
            layer_list.append(('fourier_transform_%d' %i, self.ft())) #прямое преобразование Фурье
            layer_list.append(('layer_%d' % i, torch.nn.Linear(1+dimension_list[i-1]//2, 1+dimension_list[i+1]//2).to(torch.complex64))) #комплексное линейное преобразование
            layer_list.append(('inverse_fourier_transform_%d' %i, self.ift())) #обратное преобразование Фурье
            layer_list.append(('activation_%d' % i, self.activation())) #нелинейное преобразование на выходе

        layer_list.append(('layer_%d' % (self.depth - 1), torch.nn.Linear(dimension_list[-2], dimension_list[-1]))) #нельзя сделать в цикле, потому что нет функции активации

        layerDict = OrderedDict(layer_list) #сделали упорядоченный словарь, чтобы при использовании элементы выдавались в том порядке, в котором были добавлены
        self.layers = torch.nn.Sequential(layerDict) #собственно, задали архитектуру нейросети

    def forward(self, x):
        out = self.layers(x)
        return out

наконец, PINN

In [ ]:
class PhysicsInformedNN():
    def __init__(self, X_uv, u, v, X_f, layers, lb, ub, alpha, beta):

        # граничные условия
        self.lb = torch.tensor(lb).float().to(device) #левое
        self.ub = torch.tensor(ub).float().to(device) #правое

        # данные для обучения
        self.x_uv = torch.tensor(X_uv[:, 0:1], requires_grad=True).float().to(device) #для граничных условий: (x_uv, t_uv, u, v)
        self.t_uv = torch.tensor(X_uv[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        self.v = torch.tensor(v).float().to(device)
        #self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device) #для уравнения: (x_f, t_f, f=0) (используются переданные сети точки)
        #self.t_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor((lb + (ub-lb)*lhs(2, N_f))[:, 0:1], requires_grad=True).float().to(device) #создаём N_f точек для обучения соответствия уравнению. те, что переданы, не используем
        self.t_f = torch.tensor((lb + (ub-lb)*lhs(2, N_f))[:, 1:2], requires_grad=True).float().to(device)

        #данные для законов сохранения
        self.x_fragments = 100 #разбиение при интегрировании по x при фиксированном t
        self.t_amount = 10 #в скольких точках t считается интеграл
        x_l = np.linspace(x_0, x_1, self.x_fragments).reshape(self.x_fragments,1)
        t_l = np.linspace(t_0, t_1, self.t_amount).reshape(self.t_amount,1)
        self.x_l = torch.tensor(x_l, requires_grad=True).float().to(device)
        self.t_l = torch.tensor(t_l, requires_grad=True).float().to(device)

        # числовые коэффициенты в уравнении
        self.alpha = alpha
        self.beta = beta

        # модель
        self.layers = layers
        self.dnn = FDNN(layers).to(device)

        # оптимизатор - LBFGS, обучает с точностью до 1e-5 или пока разница в точности уменьшается больше, чем точность float(?)
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(),
            lr=0.0001,
            max_iter=50000,
            max_eval=50000,
            history_size=50,
            tolerance_grad=1e-5,
            tolerance_change=1.0 * np.finfo(float).eps,
            #line_search_fn="strong_wolfe"
        )

        self.adam = torch.optim.Adam(
          self.dnn.parameters(),
          lr=0.005,
          betas=(0.9, 0.999),
          eps=1e-08,
          weight_decay=0,
          amsgrad=False)

        self.iter = 0

    def net_uv(self, x, t): # вывод модели
        u = self.dnn(torch.cat([x, t], dim=1))[:,0:1]
        v = self.dnn(torch.cat([x, t], dim=1))[:,1:2]
        return u, v

    def net_f(self, x, t): #вывод  функции
        """ The pytorch autograd version of calculating residual """
        u, v = self.net_uv(x, t)

        u_t = torch.autograd.grad(
            u, t,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0] #производая по t
        u_x = torch.autograd.grad(
            u, x,
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0] #производная по x
        u_xx = torch.autograd.grad(
            u_x, x,
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0] #вторая призводная по x

        v_t = torch.autograd.grad(
            v, t,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0] #производая по t
        v_x = torch.autograd.grad(
            v, x,
            grad_outputs=torch.ones_like(v),
            retain_graph=True,
            create_graph=True
        )[0] #производная по x
        v_xx = torch.autograd.grad(
            v_x, x,
            grad_outputs=torch.ones_like(v_x),
            retain_graph=True,
            create_graph=True
        )[0] #вторая призводная по x

        f_u = u*(u**2 + v**2)*(-self.alpha*(u**2 + v**2) + self.beta*(u**2 + v**2)**2 + 1) + u_xx - v_t #это и есть действительная и коплексная части исходного уравнения
        f_v = u_t + v*(u**2 + v**2)*(-self.alpha*(u**2 + v**2) + self.beta*(u**2 + v**2)**2 + 1) + v_xx #они получены путём подстановки q=u+i*v в него
        return f_u, f_v

    def first_law(self): #реализация первого закона сохранения
        x_step=(self.x_l[1]-self.x_l[0]).item()
        initial_integral=0
        for j in range(0,self.x_fragments): #считаем интеграл по x в точке t_0
          u, v = self.net_uv(self.x_l[j:j+1], self.t_l[0:1]) #интеграл считается от вывода нейросети
          #u, v = q_tensor(self.x_l[j:j+1], self.t_l[0:1]) #интеграл считается от q(x,t), ведь она по сути дана в t_0
          initial_integral += (u**2 + v**2) * x_step

        variation=np.zeros(self.t_amount)
        for i in range(1,self.t_amount): #сравниваем с интегралом по x в других точках
          integral=0
          for j in range(0,self.x_fragments):
            u, v = self.net_uv(self.x_l[j:j+1], self.t_l[i:i+1])
            integral += (u**2 + v**2) * x_step
          variation[i]=(initial_integral-integral)**2

        return variation.mean(axis=0) #возвращаем средний квадрат отклонения

    def second_law(self): #реализация второго закона сохранения
        x_step=(self.x_l[1]-self.x_l[0]).item()
        initial_integral=0
        for j in range(0,self.x_fragments): #считаем интеграл по x в t_0
          u, v = self.net_uv(self.x_l[j:j+1], self.t_l[0:1]) #интеграл считается от вывода нейросети
          #u, v = q_tensor(self.x_l[j:j+1], self.t_l[0:1]) #интеграл считается от q(x,t)
          u_x = torch.autograd.grad(u, self.x_l, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
          v_x = torch.autograd.grad(v, self.x_l, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
          initial_integral += ((v_x[j] * u - u_x[j] * v) * x_step)

        variation=np.zeros(self.t_amount)
        for i in range(1,self.t_amount): #сравниваем с интегралом по x в других точках
          integral=0
          for j in range(0,self.x_fragments):
            u, v = self.net_uv(self.x_l[j:j+1], self.t_l[i:i+1])
            u_x = torch.autograd.grad(u, self.x_l, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0] #почему-то градиент по self.x_l[j:j+1] посчитать не получается
            v_x = torch.autograd.grad(v, self.x_l, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0] #поэтому вычисляется градиент по self.x_l и берётся j элемент
            integral += ((v_x[j] * u - u_x[j] * v) * x_step)
          variation[i]=(initial_integral-integral)**2

        return variation.mean(axis=0) #опять возвращаем средний квадрат отклонения

    def loss_func(self): #функция потерь
        self.optimizer.zero_grad() #обнуляет градиенты

        u_pred, v_pred = self.net_uv(self.x_uv, self.t_uv)
        f_u_pred, f_v_pred = self.net_f(self.x_f, self.t_f)
        loss_uv = torch.mean(((self.u - u_pred) ** 2 + (self.v - v_pred) ** 2)/2) #средний квадрат всех отклонений от начальных условий
        loss_f = torch.mean((f_u_pred ** 2 + f_v_pred ** 2)/2) #средний квадрат всех отклонений от условия

        loss = loss_uv + loss_f

        loss.backward()
        self.iter += 1
        if self.iter % 1000 == 0:
            print('Iter %d, Loss: %.5e, Loss_uv: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_uv.item(), loss_f.item()))
            #print('loss_fl: %.5e' %loss_fl.item())
            #print('loss_sl: %.5e' %loss_sl.item())
            loss_fl = self.first_law()
            loss_sl = self.second_law()
            loss_array.append(loss.item()) #loss запоминаем для графика
            loss_fl_array.append(loss_fl.item()) #loss_fl и loss_sl тоже
            loss_sl_array.append(loss_sl.item())
            iter_array.append(self.iter)

        if self.iter % 10 == 0: #каждые 10 итераций генерируем новые точки для обучения
          random_points = lb + (ub-lb)*lhs(2, N_f)
          self.x_f = torch.tensor(random_points[:, 0:1], requires_grad=True).float().to(device)
          self.t_f = torch.tensor(random_points[:, 1:2], requires_grad=True).float().to(device)

        return loss

    def train(self): #обучение
        adam_iterations = 30000
        print('training started')
        print('%d iterations of ADAM:' %adam_iterations)
        self.dnn.train()
        for i in range(adam_iterations): #во время тренировки производится 30000 шагов adam, а дальше запускается lbfgs
            if i % 100 == 0: self.adam.param_groups[0]['lr'] = 0.99*self.adam.param_groups[0]['lr'] #экспоненциальное уменьшение шага каждые 100 шагов
            self.adam.step(self.loss_func)
        #print('LBFGS:')
        #self.optimizer.step(self.loss_func)
        #print('Total iterations: %d + %d' %(adam_iterations, (self.iter-adam_iterations)))


    def predict(self, X): #вывод нейросети и функции на входных данных
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u, v = self.net_uv(x, t)
        f_u, f_v = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        v = v.detach().cpu().numpy()
        f_u = f_u.detach().cpu().numpy()
        f_v = f_v.detach().cpu().numpy()
        return u, v, f_u, f_v

данные для обучения

In [ ]:
N_u = 200 # число точек, обучающих принимать граничные значения
N_f = 30000 # число точек, обучающих удовлетворять уравнению
x_parts = 1000 #число частей, на которые разбивается отрезок x
t_parts = 100 #число частей, на которые разбивается отрезок t
layers = [2, 100, 100, 'FL', 100, 100, 2]

x_0=-10
x_1=30
t_0=0
t_1=3.5
x = np.linspace(x_0, x_1, x_parts)
t = np.linspace(t_0, t_1, t_parts)
X, T = np.meshgrid(x, t)
Exact_u=q(X,T)[0]
Exact_v=q(X,T)[1]

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact_u.flatten()[:,None]
v_star = Exact_v.flatten()[:,None]

# границы области
lb = X_star.min(0)
ub = X_star.max(0)

# для обучения берём только данные на границах области(граничные условия по сути)
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T)) #(x,t_0)
uu1 = Exact_u[0:1,:].T
vv1 = Exact_v[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1])) #(x_0,t)
uu2 = np.zeros((t_parts,1)) #граничные условия поставил 0 в соответствии с требованиями
vv2 = np.zeros((t_parts,1))
xx3 = np.hstack((X[:,-1:], T[:,-1:])) #(x_1,t)
uu3 = np.zeros((t_parts,1)) #тут тоже
vv3 = np.zeros((t_parts,1))

X_uv_train = np.vstack([xx1, xx2, xx3]) #данные для тренировки в точках на границе
u_train = np.vstack([uu1, uu2, uu3])
v_train = np.vstack([vv1, vv2, vv3])
X_f_train = lb + (ub-lb)*lhs(2, N_f) #данные для тренировки в случайных точкам из области
X_f_train = np.vstack((X_f_train, X_uv_train)) #добавим к ним ещё и точки на границе, там f тоже 0

idx = np.random.choice(X_uv_train.shape[0], N_u, replace=False) #выберем из точек на границе только N_u
X_uv_train = X_uv_train[idx, :]
u_train = u_train[idx,:]
v_train = v_train[idx,:]

loss_array = [] #массив с loss в процессе обучения
loss_fl_array = [] #массивы с loss_fl и loss_sl
loss_sl_array = []
iter_array = [] #итерации с шагом 100

Обучение

In [ ]:
model = PhysicsInformedNN(X_uv_train, u_train, v_train, X_f_train, layers, lb, ub, alpha, beta)

In [ ]:
%%time
#штука сверху выведет итоговое время выполнения
model.train()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.array(iter_array)[::10], np.array(loss_array)[::10], color = "blue")
plt.ylim(0,5e-4)
plt.title('loss(iter)')
plt.show()

In [ ]:
plt.plot(np.array(iter_array), np.array(loss_fl_array), color = "blue")
plt.ylim(0,1)
plt.title('loss_fl(iter)')
plt.show()

In [ ]:
plt.plot(np.array(iter_array), np.array(loss_sl_array), color = "blue")
plt.ylim(0,1)
plt.title('loss_sl(iter)')
plt.show()

In [ ]:
torch.save(model, 'model_12(3).pth') #сохраняем модель

In [ ]:
u_pred, v_pred, f_u_pred, f_v_pred = model.predict(X_star) #можно ради интереса сравнить с суммой дух решений, которая строго говоря не является решением(но именно от неё и берутся начальные условия)
#mse_u = ((u_star-u_pred)**2).mean(axis=0).item()
#mse_v = ((v_star-v_pred)**2).mean(axis=0).item()
#mse_q = (((u_star**2+v_star**2)**0.5 - (u_pred**2+v_pred**2)**0.5)**2).mean(axis=0).item() #средний квадрат разности модулей
mse_f_u = ((f_u_pred)**2).mean(axis=0).item()
mse_f_v = ((f_v_pred)**2).mean(axis=0).item()
#print('MSE_u: %e, MSE_v: %e, MSE_q: %e' %(mse_u, mse_v, mse_q)) #сравнение с суммой решений, ради интереса
print('MSE_f_u: %e, MSE_f_v: %e' %(mse_f_u, mse_f_v))

Визуализация

Что выдала нейросеть:

In [ ]:
fig, ax = plt.subplots()
u_min, u_max = -np.abs(u_pred).max(), np.abs(u_pred).max()
c = ax.pcolormesh(T, X, u_pred.reshape((t_parts, x_parts)), cmap='RdBu', vmin=u_min, vmax=u_max)
ax.set_title('u_pred(x,t)')
ax.axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=ax)
#plt.scatter(X_uv_train[:,1], X_uv_train[:,0], s=7, color='black')
#plt.scatter(X_f_train[:,1], X_f_train[:,0], s=3, color='black', alpha=0.3)
plt.show()

fig, ax = plt.subplots()
v_min, v_max = -np.abs(v_pred).max(), np.abs(v_pred).max()
c = ax.pcolormesh(T, X, v_pred.reshape((t_parts, x_parts)), cmap='RdBu', vmin=v_min, vmax=v_max)
ax.set_title('v_pred(x,t)')
ax.axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=ax)
plt.show()

Что получается при суммировании решений:

In [ ]:
fig, ax = plt.subplots()
u_min, u_max = -np.abs(Exact_u).max(), np.abs(Exact_u).max()
c = ax.pcolormesh(T, X, Exact_u, cmap='RdBu', vmin=u_min, vmax=u_max)
ax.set_title('u_sum(x,t)')
ax.axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=ax)
plt.show()

fig, ax = plt.subplots()
v_min, v_max = -np.abs(Exact_v).max(), np.abs(Exact_v).max()
c = ax.pcolormesh(T, X, Exact_v, cmap='RdBu', vmin=v_min, vmax=v_max)
ax.set_title('v_sum(x,t)')
ax.axis([t_0, t_1, x_0, x_1])
fig.colorbar(c, ax=ax)
plt.show()

Решение в срезах по t:

In [ ]:
t=t_0
while t<=t_1:
  current_row = int(t_parts*(t-t_0)/(t_1-t_0))
  if t==t_1: current_row=-1 #если смотрим значения в t_1, то ясно что это последняя колонка
  plt.plot(X[current_row, :], u_pred.reshape((t_parts, x_parts))[current_row,:], color = "red") #предсказанное решение
  #plt.plot(X[current_row, :], Exact_u[current_row,:], color = "green") #суммированное решение
  plt.title('u_pred(x,t=%.2f)'%t)
  plt.show()
  t+=abs(t_1-t_0)/7

In [ ]:
t=t_0
while t<=t_1:
  current_row = int(t_parts*(t-t_0)/(t_1-t_0))
  if t==t_1: current_row=-1 #если смотрим значения в t_1, то ясно что это последняя колонка
  plt.plot(X[current_row, :], v_pred.reshape((t_parts, x_parts))[current_row,:], color = "red")
  #plt.plot(X[current_row, :], Exact_v[current_row,:], color = "green")
  plt.title('v_pred(x,t=%.2f)'%t)
  plt.show()
  t+=abs(t_1-t_0)/7

In [ ]:
t=t_0
while t<=t_1:
  current_row = int(t_parts*(t-t_0)/(t_1-t_0))
  if t==t_1: current_row=-1 #если смотрим значения в t_1, то ясно что это последняя колонка
  q_abs_pred = (u_pred.reshape((t_parts, x_parts))[current_row,:]**2 + v_pred.reshape((t_parts, x_parts))[current_row,:]**2)**0.5
  q_abs_exact = (Exact_u[current_row,:]**2 + Exact_v[current_row,:]**2)**0.5
  plt.plot(X[current_row, :], q_abs_pred, color = "red")
  #plt.plot(X[current_row, :], q_abs_exact, color = "green")
  plt.title('q_abs(x,t=%.2f)'%t)
  plt.show()
  t+=abs(t_1-t_0)/7

Точность выполнения условий уравнения:

In [ ]:
loss_abs_f = (f_u_pred**2 + f_v_pred**2)**0.5 #тензор модулей значений уравнения
loss_abs_f_array=[]
t_array=[]
for t in np.linspace(t_0, t_1, 100):
  current_row = int(t_parts*(t-t_0)/(t_1-t_0))
  if t==t_1: current_row=-1 #если смотрим значения в t_1, то ясно что это последняя колонка
  loss_abs_f_array.append(loss_abs_f.reshape((t_parts, x_parts))[current_row,:].mean())
  t_array.append(t)
plt.figure(figsize=(10,4))
plt.plot(t_array, loss_abs_f_array, color = "blue")
plt.title('mean_x(|f(t)|)') #среднее значение модуля функции на срезе при фиксированном t
plt.show()
print('Mean_xt(|f|): %e' %loss_abs_f.mean(axis=0).item()) #среднее значение модуля функции на всей сетке

Проверка законов сохранения:

In [ ]:
import matplotlib.ticker as mtick
x_fragments = 400 #разбиение при интегрировании по x при фиксированном t
t_amount = 10 #в скольких точках t считается интеграл
x_l = np.linspace(x_0, x_1, x_fragments).reshape(x_fragments,1)
t_l = np.linspace(t_0, t_1, t_amount).reshape(t_amount,1)
X_l = torch.tensor(x_l, requires_grad=True).float().to(device)
T_l = torch.tensor(t_l, requires_grad=True).float().to(device)
x_step=(X_l[1]-X_l[0]).item()

#реализация первого закона сохранения
u, v = model.net_uv(X_l, torch.full((x_fragments,1),T_l[0:1].item(), requires_grad=True).float().to(device))
initial_integral = ((u**2 + v**2) * x_step).sum() #вычисляем интеграл в t_0
u, v = model.net_uv(X_l.expand(x_fragments, t_amount).T.reshape(x_fragments*t_amount,1), T_l.expand(t_amount, x_fragments).reshape(x_fragments*t_amount,1))
first_variation = (((u.reshape(t_amount, x_fragments)**2 + v.reshape(t_amount, x_fragments)**2) * x_step).sum(axis=1) - initial_integral)**2 #вычисляем интегралы при других t и находим отклонение
first_variation = first_variation.detach()
MSE_fl = first_variation.mean(axis=0).item() #возвращаем средний квадрат отклонения
mean_fl = (first_variation**0.5).mean(axis=0).item() #а также средний модуль отклонения
plt.figure(figsize=(10,4))
plt.plot(t_l, (first_variation**0.5/initial_integral).detach().cpu(), color = "blue") #строим график соблюдения первого закона сохранения(чем ближе к 0, тем лучше)
plt.title('|first_int(t)-first_int(t=0)|/first_int(t=0)')
plt.ylim(0,1)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0)) #меняем числа на проценты
plt.show()

#реализация второго закона сохранения
u, v = model.net_uv(X_l, torch.full((x_fragments,1),T_l[0:1].item(), requires_grad=True).float().to(device))
u_x = torch.autograd.grad(u, X_l, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
v_x = torch.autograd.grad(v, X_l, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
initial_integral = ((v_x*u - u_x*v) * x_step).sum() #вычисляем интеграл в t_0
X_l_expanded = X_l.expand(x_fragments, t_amount).T.reshape(x_fragments*t_amount,1)
T_l_expanded = T_l.expand(t_amount, x_fragments).reshape(x_fragments*t_amount,1)
u, v = model.net_uv(X_l_expanded, T_l_expanded)
u_x = torch.autograd.grad(u, X_l_expanded, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
v_x = torch.autograd.grad(v, X_l_expanded, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
second_variation = ((((v_x*u).reshape(t_amount, x_fragments) - (u_x*v).reshape(t_amount, x_fragments)) * x_step).sum(axis=1) - initial_integral)**2 #вычисляем интегралы при других t и находим отклонение
second_variation = second_variation.detach()
MSE_sl = second_variation.mean(axis=0).item() #возвращаем средний квадрат отклонения
mean_sl = (second_variation**0.5).mean(axis=0).item() #а также средний модуль отклонения
plt.figure(figsize=(10,4))
plt.plot(t_l, (second_variation**0.5/initial_integral).detach().cpu(), color = "blue") #строим график соблюдения первого закона сохранения(чем ближе к 0, тем лучше)
plt.title('|second_int(t)-second_int(t=0)|/second_int(t=0)')
plt.ylim(0,1)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
plt.show()

print('First law MSE: %e, Second law MSE: %e' %(MSE_fl, MSE_sl))
print('First law mean: %e, Second law mean: %e' %(mean_fl, mean_sl)) #среднее значение модуля отклонения интеграла от первоначального значения